In [47]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [57]:
try:
    from StringIO import BytesIO
except ImportError:
    from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

In [60]:
url = "https://docs.google.com/spreadsheets/d/1wfYdPJkaPFHj_WW1LhTL46fosN92fhEnkrfeLVYbkxk/edit?usp=sharing"
response = urlopen(url)
income = response.read()
read = csv.DictReader(income)

In [59]:
income

b'<!DOCTYPE html><html lang="el"><head><meta property="og:title" content="adult"><meta property="og:type" content="article"><meta property="og:site_name" content="Google Docs"><meta property="og:url" content="https://docs.google.com/spreadsheets/d/1wfYdPJkaPFHj_WW1LhTL46fosN92fhEnkrfeLVYbkxk/edit?usp=sharing&amp;usp=embed_facebook"><meta property="og:image" content="https://lh3.googleusercontent.com/DOtjwO6dTT31aMp5asOI7WYKFxtl6OQVqHs9h5vZhjktbwB-tb_Og2YZ6kyo75egobzAoQ=w1200-h630-p"><meta property="og:image:width" content="1200"><meta property="og:image:height" content="630"><meta property="og:description" content="adult\n\nage,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income\n90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,&lt;=50K\n82,Private,132870,HS-grad,9,Widowed,Exec-manag..."><meta name="google" content="notranslate"><meta http-equiv="X-UA-Com